In [1]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import pandas as pd

dataset = load_dataset("zjunlp/FactCHD")

In [2]:
df = pd.DataFrame(dataset["train"])
print(df.columns)

Index(['id', 'label', 'query', 'response', 'reason', 'type', 'domain',
       'subgraph_type', 'comparison_size', 'set_size', 'answer_size'],
      dtype='object')


In [16]:
def wrap_response(df):
    df['response'] = ' <false> ' + df['response'] + ' </false> '

In [19]:
# Get 29,768 examples (where domain is common_wikipedia or common_wikidata)
filtered_df = df[df['domain'].isin(['common_wikipedia', 'common_wikidata'])]

# Get 500 examples of true query-response pairs
true = filtered_df[df['label'].isin(['FACTUAL'])]
true_trim = true[['label', 'query', 'response']]
true_trim['label'] = true_trim['label'].replace('FACTUAL', 'TRUE')
true_sample = true_trim.sample(n=500, replace=False, random_state=1) # same sample everytime (remove random_state to get different samples everytime) 

# Get 500 examples of false query-response pairs
false = filtered_df[df['label'].isin(['NON-FACTUAL'])]
false_trim = false[['label', 'query', 'response']]
false_trim['label'] = false_trim['label'].replace('NON-FACTUAL', 'FALSE')
false_sample = false_trim.sample(n=500, replace=False, random_state=1) # same sample everytime (remove random_state to get different samples everytime)

# Combine and shuffle (just cuz) to get 1000 training examples
combined_df = pd.concat([true_sample, false_sample], ignore_index=True)
shuffled_df = combined_df.sample(frac=1, random_state=1)

wrap_response(shuffled_df)
shuffled_df = shuffled_df.astype({'label': 'string', 'query': 'string', 'response': 'string'})
shuffled_df.to_csv('factCHD.csv', index=False)

print(shuffled_df['response'])

507     <false> Among the given states and sovereign ...
818     <false> Passengers grossed $303 million when ...
452     <false> Matt Damon starred in a biopic. </fal...
368     <false> The total fertility rate of the Unite...
242     <false> Mike Pence served as the chairman of ...
                             ...                        
767     <false> Anne Sullivan was born on April 12 18...
72      <false> "Emma" is a film based on literature ...
908     <false> Among the given places, *Geneva* has ...
235     <false> Among the given sovereign states, *Ru...
37      <false> The population of the San Francisco B...
Name: response, Length: 1000, dtype: string


/tmp/ipykernel_395614/656176293.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  true = filtered_df[df['label'].isin(['FACTUAL'])]
/tmp/ipykernel_395614/656176293.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_trim['label'] = true_trim['label'].replace('FACTUAL', 'TRUE')
/tmp/ipykernel_395614/656176293.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  false = filtered_df[df['label'].isin(['NON-FACTUAL'])]
/tmp/ipykernel_395614/656176293.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta